# Introduction

Python의 Model I/O에 대한 학습함

# FewShotPromptTemplate


In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# 예시를 주고 그것을 기반으로 답변을 할 수있게 함

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [23]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}") 

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [24]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI:  
I know this:  
Capital: Madrid  
Language: Spanish  
Food: Paella and Tapas  
Currency: Euro  

AIMessage(content='AI:  \nI know this:  \nCapital: Madrid  \nLanguage: Spanish  \nFood: Paella and Tapas  \nCurrency: Euro  ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--e519cd6f-d363-443b-8958-da904c5ae503-0')

# FewShotChatMessagePromptTemplate

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [26]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

In [27]:
chain = final_prompt | chat
chain.invoke(
    {
        "country" : "Korea"
     }
)


        I know this:
        Divided into two countries: North Korea and South Korea
        Capital of North Korea: Pyongyang
        Capital of South Korea: Seoul
        Language: Korean
        Currency: North Korean Won (North) and South Korean Won (South)
        

AIMessage(content='\n        I know this:\n        Divided into two countries: North Korea and South Korea\n        Capital of North Korea: Pyongyang\n        Capital of South Korea: Seoul\n        Language: Korean\n        Currency: North Korean Won (North) and South Korean Won (South)\n        ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--a2e0edbe-45a2-4e21-92df-271730baef82-0')

# LengthBaseExampleSelector

In [28]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI (
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],

)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [29]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)

In [30]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Brazil")

In [31]:
chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: Here is what I know about Spain:

- Capital: Madrid
- Language: Spanish (Castilian)
- Food: Tapas, Paella, Jamón Ibérico
- Currency: Euro
- Notable Features: Famous for its art, architecture (like Gaudí's Sagrada Família), and vibrant festivals (such as La Tomatina and Running of the Bulls).

AIMessage(content="AI: Here is what I know about Spain:\n\n- Capital: Madrid\n- Language: Spanish (Castilian)\n- Food: Tapas, Paella, Jamón Ibérico\n- Currency: Euro\n- Notable Features: Famous for its art, architecture (like Gaudí's Sagrada Família), and vibrant festivals (such as La Tomatina and Running of the Bulls).", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--267246d9-58a4-4fed-a72c-5a878600cda4-0')

# RandomBaseExampleSelector

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [33]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [34]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

In [35]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialization and Composition

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

prompt = load_prompt("./prompt.json")

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [43]:
chain = prompt | chat

chain.invoke({
    "country": "Germmany"
})

The capital of Germany is Berlin.

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--70accca2-8124-4ff2-aeb4-77ab112fd293-0')

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

In [46]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    this is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

/var/folders/mh/rl6y80hd7tq9c91b7bx47m640000gn/T/ipykernel_77711/1909309788.py:42: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  full_prompt = PipelinePromptTemplate(


Arrrr, matey! Me favorite grub be a hearty plate o' salted fish and hardtack, washed down with a tankard o' grog! Nothin' like a good meal to keep a pirate's spirit high! Arg arg!

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty plate o' salted fish and hardtack, washed down with a tankard o' grog! Nothin' like a good meal to keep a pirate's spirit high! Arg arg!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--98fbd0d0-9eef-4c3e-898b-18c5dc5481c3-0')

# Cashing

In [53]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)


chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

In [54]:
chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

In [55]:
chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

# Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False)

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)
    print(usage.total_cost) # 전체 비용
    print(usage.total_tokens) # 토큰 총합
    print(usage.prompt_tokens) # 프롬프트에 사용된 토큰 수
    print(usage.completion_tokens) # 모델이 쓴 토큰

Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')


Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
0.0
0
0
0
